In [1]:
!pip install rwkv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 80.9 MB/s eta 0:00:00


In [2]:
import os
# set these before import RWKV
os.environ['RWKV_JIT_ON'] = '1'
os.environ["RWKV_CUDA_ON"] = '0'

In [3]:
!wget https://huggingface.co/BlinkDL/rwkv-4-pile-3b/resolve/main/RWKV-4-Pile-3B-20220910-165.pth

--2023-04-15 11:47:49--  https://huggingface.co/BlinkDL/rwkv-4-pile-3b/resolve/main/RWKV-4-Pile-3B-20220910-165.pth
Resolving huggingface.co (huggingface.co)... 13.224.249.63, 13.224.249.86, 13.224.249.35, ...
Connecting to huggingface.co (huggingface.co)|13.224.249.63|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/83/3c/833ca8226054ae3c90a5d426f6de2c70a1ae3dc3b9f525298cab3f745a491251/5607297fd260ac63e813b45e5b1162e2259db968a265dc9e43ed365ae660e360?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27RWKV-4-Pile-3B-20220910-165.pth%3B+filename%3D%22RWKV-4-Pile-3B-20220910-165.pth%22%3B&Expires=1681818470&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzgzLzNjLzgzM2NhODIyNjA1NGFlM2M5MGE1ZDQyNmY2ZGUyYzcwYTFhZTNkYzNiOWY1MjUyOThjYWIzZjc0NWE0OTEyNTEvNTYwNzI5N2ZkMjYwYWM2M2U4MTNiNDVlNWIxMTYyZTIyNTlkYjk2OGEyNjVkYzllNDNlZDM2NWFlNjYwZTM2MD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l

In [4]:
!git clone https://github.com/BlinkDL/ChatRWKV

Cloning into 'ChatRWKV'...
remote: Enumerating objects: 1323, done.
remote: Counting objects: 100% (199/199), done.
remote: Compressing objects: 100% (139/139), done.
remote: Total 1323 (delta 75), reused 149 (delta 45), pack-reused 1124
Receiving objects: 100% (1323/1323), 26.97 MiB | 17.91 MiB/s, done.
Resolving deltas: 100% (709/709), done.


In [5]:
from rwkv.model import RWKV
from rwkv.utils import PIPELINE, PIPELINE_ARGS

# download models: https://huggingface.co/BlinkDL
model = RWKV(model='/content/RWKV-4-Pile-3B-20220910-165.pth', strategy='cuda fp16')
pipeline = PIPELINE(model, "/content/ChatRWKV/20B_tokenizer.json") # 20B_tokenizer.json is in https://github.com/BlinkDL/ChatRWKV


RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading /content/RWKV-4-Pile-3B-20220910-165.pth ...
Strategy: (total 32+1=33 layers)
* cuda [float16, float16], store 33 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 25-cuda-float16-float16 26-cuda-float16-float16 27-cuda-float16-float16 28-cuda-float16-float16 29-cuda-float16-float16 30-cuda-float16-float16 31-cuda-float16-float16 32-cuda-float16-float16 
emb.weight                        f16     

In [9]:
ctx = "Here's a short story. One day Chris took his dog to a garden.\nThe Story:"
print(ctx, end='')

def my_print(s):
    print(s, end='', flush=True)

# For alpha_frequency and alpha_presence, see "Frequency and presence penalties":
# https://platform.openai.com/docs/api-reference/parameter-details

args = PIPELINE_ARGS(temperature = 1, top_p = 0.7, top_k = 50, # top_k = 0 then ignore
                     alpha_frequency = 0.5,
                     alpha_presence = 0.5,
                     token_ban = [0], # ban the generation of some tokens
                     token_stop = [], # stop generation whenever you see any token here
                     chunk_len = 256) # split input into chunks to save VRAM (shorter -> slower)

pipeline.generate(ctx, token_count=150, args=args, callback=my_print)
print('\n')

Here's a short story. One day Chris took his dog to a garden.
The Story:
Chris sat on the garden bench with his dog, Mica. The sun was hot
and Chris was sitting in the shade of a big old tree. He had his dog
in his lap and he'd told Mica about being outside all day with no sun.
He told her about some of the trees he knew from before he left for
college and then remembered how much they looked like real trees, but
what they didn't have was leaves and grass and bushes to make them look
like flowers. After that, he started thinking about the many things that
could be lost when people leave their dogs outside for a while. And then,
he thought of this song: "Happiness is a beautiful thing."

